In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
print("Tensorflow version " + tf.__version__)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

Tensorflow version 2.8.0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.127.243.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.127.243.202:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [30]:
import numpy as np
import keras
import time
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback
from keras.layers import Dense,Reshape,Input,BatchNormalization,Dropout,concatenate,Flatten,Lambda,Resizing,CuDNNLSTM,concatenate,Layer,LSTM
from keras import  Model,layers
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split

In [ ]:
def find_removable(n,batch_size=64):
      return n%batch_size

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/OAD/OAD/Vincent/BASE/40/'
# base_path = '/content/drive/MyDrive/Colab Notebooks/NTU/Vincent/ntu_3cam/'
input_shape= (25,40,3)

train_x = np.load(base_path+'train_x.npy')
test_x = np.load(base_path+'test_x.npy')
train_y = np.load(base_path+'train_y.npy')
test_y = np.load(base_path+'test_y.npy')

batch_size=64

n = find_removable(len(train_x),batch_size)
train_x = train_x[:-n]
train_y = train_y[:-n]

print('lost',n)

train = (train_x,train_y)
test = (test_x,test_y)

lost 31


In [49]:
with strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  inp=Input(np.asarray(input_shape))
  output = Resizing(64, 64)(inp)
  model = VGG16(include_top=False,input_shape=(64,64,3))
  for layer in model.layers:
      layer.trainable = True
  output = model(output)
  output = Flatten()(output)
  output = Dense(2048)(output)
  output = BatchNormalization()(output)
  output = Dense(10, activation='softmax',kernel_regularizer='l1')(output)
  model = Model(inp, output)
  model.summary()
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=optimizer,
                metrics=['accuracy'])

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_36 (InputLayer)       [(None, 25, 40, 3)]       0         
                                                                 
 resizing_18 (Resizing)      (None, 64, 64, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_17 (Flatten)        (None, 2048)              0         
                                                                 
 dense_34 (Dense)            (None, 2048)              4196352   
                                                                 
 batch_normalization_6 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                          

In [50]:
filepath ="oad.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
callbacks = [checkpoint,EarlyStopping(monitor='val_accuracy', patience=100,)]
debut = time.time()
history=model.fit(train[0], train[1],
              epochs=200,
              batch_size=batch_size,
              validation_data=(test[0],test[1]),
              callbacks=callbacks,
              verbose=1)
fin = time.time()
print("Temps total : " + str(fin - debut) + "ms")
model = keras.models.load_model('oad.hdf5')
keras.models.save_model(model, base_path+'_vgg16_64.h5')
print(model.evaluate(test[0],test[1]))

Epoch 1/200
186/186 [==============================] - 16s 58ms/step - loss: 4.3747 - accuracy: 0.8487 - val_loss: 3.9978 - val_accuracy: 0.5760
Epoch 2/200
186/186 [==============================] - 7s 38ms/step - loss: 1.4995 - accuracy: 0.9873 - val_loss: 1.3882 - val_accuracy: 0.7637
Epoch 3/200
186/186 [==============================] - 7s 38ms/step - loss: 0.5621 - accuracy: 0.9959 - val_loss: 0.7698 - val_accuracy: 0.8652
Epoch 4/200
186/186 [==============================] - 8s 41ms/step - loss: 0.4412 - accuracy: 0.9973 - val_loss: 0.6974 - val_accuracy: 0.8714
Epoch 5/200
186/186 [==============================] - 7s 38ms/step - loss: 0.4022 - accuracy: 0.9992 - val_loss: 0.5733 - val_accuracy: 0.9229
Epoch 6/200
186/186 [==============================] - 7s 38ms/step - loss: 0.3957 - accuracy: 0.9979 - val_loss: 0.5331 - val_accuracy: 0.9280
Epoch 7/200
186/186 [==============================] - 7s 38ms/step - loss: 0.3814 - accuracy: 0.9982 - val_loss: 0.5111 - val_accuracy